# 模块化目标

## 对原有项目做抽象/一般化
- prj_name: "中国报业首届盈利模式创新大赛"
- prj_src：["微信公众号", "传媒融中对"]
- prj_lang："zh_CN"  # 中文
- process: ["LDA", "01"]

## 对数据之输入及输出做最小化设计
### 原来
- filename: "word_freq_new": "data_sets/submissions_profit_model_word_freq_new.xlsx"
- sheetname: "dataset"
- df name: dataset_df
- 必需/关键栏位['BOW_JB', 'lentity', 'lcontent']
    - dataset_df.BOW_JB 
        - 小小研究 keyword in context
    - dataset_df.lcontent
        - 建立字频表：先tokenize 再 freq/rank     
        - 合并字词
        - 移除单字词
- 额外/元数据栏位[label0...label5]不同文本结点的内容, 信息可视化要用到, 来源於元数据或entity words，可考虑抽出来做额外脚本及模块化
    - dataset_df.numb 所属报业集团 title lentity
- 主题建模处理
    - dataset_df.lcontent
    - 建立corpus
    - 产出doc_term_dists
    - _model_['doc_topic_dists']  = lda.transform(doc_term_dists)
        - 输入 doc_term_dists  文本-字词分布矩阵
        - 输出 model  LDA主题模型
- 随机森林
    * 还没应用, 纯实验
    * 输入 doc_term_dists 文本-字词分布矩阵
    * 产出 df_importance 可供助理/研究人员建构taxonomy使用
    
    
### 新版
- 预处理 (未完成)
- 主题建模处理 (已完成, 廖汉腾, 7 小时工时)
    - filename: "LDA_input.csv"
    - df name: df
    - 必需/关键栏位['doc_label', 'doc_content']
    - 产出tsv
        - "LDA_topic_term_n": "data_sets/LDA_topic_term_{n}.tsv",     # output
        - "LDA_doc_topic_n": "data_sets/LDA_doc_topic_{n}.tsv",       # output
    - 产出pkl模型
        - "LDA_pickle_n": "pkl/LDA_{n}.pickle"
- 随机森林 (未完成)
    
### 模块化分析

应该采 原内容丶实体丶无实体内容 三种数据流拆分处理

- 主题模型的运算，仅做无实体内容
- 主题模型后的文本标记label，可采用(1)实体内容或(2)其它元数据内容，应该为另外额外模组/块
- 原内容如何变成实体及无实体内容，应该是在以前就处理完毕
- 主题模型的核心模块，应该简化
  - 中英文均能处理，中文内容需要先补上空白
  - 输出LDA 模型，可以批量



In [1]:
fn = { "LDA_input": "data_sets/LDA_input.csv",# input
       "LDA_pickle_n": "pkl/LDA_{n}.pickle",  # output
       "LDA_topic_term_n": "data_sets/LDA_topic_term_{n}.tsv",     # output
       "LDA_doc_topic_n": "data_sets/LDA_doc_topic_{n}.tsv",       # output
      }

In [2]:
import pandas as pd
from collections import OrderedDict, defaultdict
import pickle

# 输入简单文本数据
必要文本doc栏位，最少
* doc_label
* doc_content

In [3]:
df = pd.read_csv(fn["LDA_input"], sep="\t", encoding="utf8", index_col=0)
corpus = df.doc_content.to_dict(into=OrderedDict)  # 从to_list() 升级改成to_dict()

# 使用LDA核心模塊
* .doc_vectorizer # 文本矢量化 LDA不需用tf-idf，用tf即可
* .LDA_run # LDA主题模型 指定要几个主题，預設值N=7
* .gen_df_topic    # 产出Topics主题的表格
* .gen_df_doc_topic # 产出最简化的文本-主题-赋值矩阵

In [4]:
import LDA_core as LDAc

In [5]:
%%time

# 测一个
min_occurences = 4 # 特徵值门槛
no_topics = 7
N, LDA,topic_term, doc_topic = LDAc.LDA_run_all(corpus, no_topics, 
                                                min_df=min_occurences/len(corpus)) 
                                              #, max_df=0.5 , max_features=50000

Wall time: 6.92 s


-----
# 批次输出模型

## 纯模型, 无 doc label

In [6]:
for N in range(4,13):
    _model_, LDA,topic_term, doc_topic = LDAc.LDA_run_all(corpus, N, min_df=min_occurences/len(corpus))
    
    with open(fn["LDA_pickle_n"].format(n=N), 'wb') as handle:
        pickle.dump(_model_, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    topic_term.to_csv(fn["LDA_topic_term_n"].format(n=N), sep="\t", encoding="utf8")
    doc_topic.to_csv(fn["LDA_doc_topic_n"].format(n=N), sep="\t", encoding="utf8")
        